## FP Growth算法

用Apriori算法寻找频繁项集(Frequent itemset)常常被认为有两个缺点，其一是计算速度慢，其二是计算成本高。为了克服这些缺点，FP Growth 算法被提出。

它基于特定的 FP Tree 数据结构实现，构建 FP Tree 只需要对数据库进行两次扫描。第一次扫描是对所有元素项的出现次数进行计数，丢弃支持度小于阈值的非频繁项，得到频繁项集，并对频繁项集按照支持度的递减顺序排序。第二次扫描则是在构建 FP Tree，从空集开始，依次读入排序好的频繁项集中各条事务。如果树中已存在现有元素，则增加现有元素的值；如果现有元素不存在，则向树添加一个分枝。接下来寻找频繁项集只需要对 FP Tree 中的元素计算前缀路径即可。

### 1. 载入模块

In [28]:
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import mlxtend
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules

### 2. 载入数据集
由于该数据集每一行都是市场数据，故不需要header行。如果这里保留了header行，后续有的代码会执行非常慢。

In [29]:
dataset = pd.read_csv("Market_Basket_Optimisation.csv", header=None)
dataset.shape

(7501, 20)

In [30]:
dataset.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 3. 将所有购买的物品存储在 numpy 数组中

In [31]:
transaction = []
for i in range(0, dataset.shape[0]):
    for j in range(0, dataset.shape[1]):
        transaction.append(dataset.values[i,j])
transaction = np.array(transaction)
print(transaction)

['shrimp' 'almonds' 'avocado' ... 'nan' 'nan' 'nan']


### 4. 计算各个元素的出现个数

In [32]:
df = pd.DataFrame(transaction, columns=["items"])
df["incident_count"] = 1
indexNames = df[df['items'] == 'nan'].index
df.drop(indexNames, inplace=True)
df_table = df.groupby('items').sum().sort_values("incident_count", ascending=False).reset_index()
df_table.head(5).style.background_gradient(cmap="Blues")

,items,incident_count
0,mineral water,1788
1,eggs,1348
2,spaghetti,1306
3,french fries,1282
4,chocolate,1230


In [33]:
df_table["all"] = "Top 50 items" 
fig = px.treemap(df_table.head(50), path=['all', "items"], values='incident_count',
                  color=df_table["incident_count"].head(50), hover_data=['items'],
                  color_continuous_scale='Blues',
                )
fig.show()

### 5. 调节 Dataset 的格式，使之转化为便于绘制 FP Tree 的格式

表格中的元素均为 True 或 False，表示该行数据中包含的商品信息。

In [34]:
transaction = []
for i in range(dataset.shape[0]):
    transaction.append([str(dataset.values[i,j]) for j in range(dataset.shape[1])])
transaction = np.array(transaction)
te = TransactionEncoder()
te_ary = te.fit(transaction).transform(transaction)
dataset = pd.DataFrame(te_ary, columns=te.columns_)
dataset.head()

,asparagus,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,black tea,blueberries,...,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,False,True,True,False,True,False,False,False,False,False,...,False,True,False,False,True,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


### 6. 选择其中 30 个特征

完整的数据集有 121 个特征，直接从中提取频繁项耗费时间较长，故我们只选择其中 30 个特征。

In [35]:
first30 = df_table["items"].head(30).values 
dataset = dataset.loc[:,first30] 
dataset.shape

(7501, 30)

### 7. 调用 FP Growth 算法

我们规定频繁项的 Support 值必须不低于 0.05.

In [36]:
res=fpgrowth(dataset,min_support=0.05, use_colnames=True)
res

,support,itemsets
0,0.238368,(mineral water)
1,0.132116,(green tea)
2,0.076523,(low fat yogurt)
3,0.071457,(shrimp)
4,0.065858,(olive oil)
5,0.063325,(frozen smoothie)
6,0.179709,(eggs)
7,0.087188,(burgers)
8,0.062525,(turkey)
9,0.129583,(milk)


接下来进行关联规则的寻找，准则为提升度 (lift)，min_threshold=1 意味着要求提升度大于 1 才可以被认为是频繁出现的关联规则。

In [37]:
res=association_rules(res, metric="lift", min_threshold=1)
res

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(mineral water),(eggs),0.238368,0.179709,0.050927,0.213647,1.188845,0.008090,1.043158,0.208562
1,(eggs),(mineral water),0.179709,0.238368,0.050927,0.283383,1.188845,0.008090,1.062815,0.193648
2,(spaghetti),(mineral water),0.174110,0.238368,0.059725,0.343032,1.439085,0.018223,1.159314,0.369437
3,(mineral water),(spaghetti),0.238368,0.174110,0.059725,0.250559,1.439085,0.018223,1.102008,0.400606
4,(mineral water),(chocolate),0.238368,0.163845,0.052660,0.220917,1.348332,0.013604,1.073256,0.339197
5,(chocolate),(mineral water),0.163845,0.238368,0.052660,0.321400,1.348332,0.013604,1.122357,0.308965


以置信度 (Confidence) 进行排序。

In [38]:
res.sort_values("confidence",ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
2,(spaghetti),(mineral water),0.174110,0.238368,0.059725,0.343032,1.439085,0.018223,1.159314,0.369437
5,(chocolate),(mineral water),0.163845,0.238368,0.052660,0.321400,1.348332,0.013604,1.122357,0.308965
1,(eggs),(mineral water),0.179709,0.238368,0.050927,0.283383,1.188845,0.008090,1.062815,0.193648
3,(mineral water),(spaghetti),0.238368,0.174110,0.059725,0.250559,1.439085,0.018223,1.102008,0.400606
4,(mineral water),(chocolate),0.238368,0.163845,0.052660,0.220917,1.348332,0.013604,1.073256,0.339197
0,(mineral water),(eggs),0.238368,0.179709,0.050927,0.213647,1.188845,0.008090,1.043158,0.208562
